In [1]:
from makeit.retrosynthetic.mcts.tree_builder import MCTS
from makeit.synthetic.evaluation.tree_evaluator import TreeEvaluator
from makeit.utilities.io.logger import MyLogger
import makeit.global_config as gc
import pandas as pd
from makeit.utilities.io import name_parser
from rdkit import Chem
from sklearn.preprocessing import MultiLabelBinarizer
import pickle
import pprint
# from construct_path_chemical_matrix import get_chemical_table_from_routes

MyLogger.initialize_logFile()

Using Theano backend.
/home/hanyug/.conda/envs/my_askcos/lib/python2.7/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/home/hanyug/.conda/envs/my_askcos/lib/python2.7/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def get_chemical_table_from_routes(routes, contexts):
    # all_chemicals = get_all_chemicals_in_all_routes(routes)
    # print(all_chemicals)
    # print(len(all_chemicals))
    all_routes_chemical_only = []
    all_routes_target = []
    all_routes_score = []
    for route in routes:
        route_target = route['chemicals'][0]['SMILES']
        route_chemical_only = [chemical['SMILES'] for chemical in route['chemicals'] if chemical['is_starting']]
        all_routes_chemical_only.append(route_chemical_only)
        all_routes_target.append(route_target)
        all_routes_score.append(route['score'])
    ohe = MultiLabelBinarizer()
    ohe_context = MultiLabelBinarizer()
    # ohe.fit(all_chemicals)
    # print('transformed')
    all_routes_chemical_onehot = ohe.fit_transform(all_routes_chemical_only)
    all_routes_context_onehot = ohe_context.fit_transform(contexts)
    chemical_smiles = list(ohe.classes_)
    context_smiles = list(ohe_context.classes_)
    return (all_routes_target, all_routes_chemical_onehot, chemical_smiles,all_routes_score, all_routes_context_onehot, context_smiles)

def get_chemicals_from_tree(tree, path = [],):
    path.append({'SMILES':tree['smiles'],
                 'is_starting': False})
    if not tree['children']:
        path[-1]['is_starting']=True
        pass
    else:
        for child in tree['children']:
            [get_chemicals_from_tree(child_chem, path) for child_chem in child['children']]
    return path

def get_context_from_tree(tree, path = [],):
    if not tree['children']:
        pass
    else:
        for child in tree['children']:
            context = child['context']
            [path.append(context[i]) for i in [1,2,3] if context[i]!='']
            [get_context_from_tree(child_chem, path) for child_chem in child['children']]
    return list(set(path))


print('start to load targets')
# dataset = pd.read_csv('common_substructure.csv')
# dataset = pd.read_csv('Top30DrugsByPrescription.csv')
# dataset.head()

# with open('truncated_mol_lib.pickle','r') as MOLLIB:
with open('WHO_EM/WHO_truncated.pickle','r') as MOLLIB:
    truncated_mol_lib = pickle.load(MOLLIB)


print(len(truncated_mol_lib))# for name in dataset['SMILES']:
#     try:
#         mol = name_parser.name_to_molecule(name)
#         mol_lib.append(mol)
#     except:
#         continue
print("{} target molecules to expand".format(len(truncated_mol_lib)))
celery = False
NCPUS = 4
# treeBuilder = TreeBuilder(celery=celery, mincount=25, mincount_chiral=10)
Tree = MCTS(nproc=NCPUS, mincount=gc.RETRO_TRANSFORMS_CHIRAL['mincount'], 
        mincount_chiral=gc.RETRO_TRANSFORMS_CHIRAL['mincount_chiral'],
        celery=celery)
treeEvaluator = TreeEvaluator(context_recommender=gc.neural_network, celery=celery)
all_routes = []
all_trees = []
all_routes_contexts = []
index_list_all_routes = []
is_first_target = True
status = 0
results_folder = "/home/hanyug/Combined_synthesis_planning/pathways/"

status_file = open(results_folder+'status for molecules.dat','w')
status_file.write('SMILES\tnumberofpaths\tnotes\n')

start to load targets
127
127 target molecules to expand
INFO@chemhistorian       : [5.293s]	Loading chemhistorian from file...
INFO@mcts_tree_builder   : [25.236s]	Doing a hard worker reset
INFO@model_loader        : [25.655s]	Loading retro synthetic template database...
INFO@retro_transformer   : [25.655s]	Loading retro-synthetic transformer, including all templates with more than 10 hits (5 for chiral reactions)
INFO@template_transformer: [25.656s]	Loading templates from /home/hanyug/Make-It/makeit/data/local_db_dumps/retrotransformer_chiral_using_reaxys_v2-transforms_retro_v9_mincount10_mincountchiral5.pkl
INFO@template_transformer: [341.548s]	Loaded templates. Using 163723 templates
INFO@retro_transformer   : [341.724s]	Retrosynthetic transformer has been loaded - using 163723 templates.
INFO@model_loader        : [341.727s]	Retro synthetic transformer loaded.
INFO@fast_filter         : [341.732s]	Starting to load fast filter
INFO@fast_filter         : [354.709s]	Done loading fast

In [4]:
for mol in truncated_mol_lib[:3]:
# for mol in [Chem.MolFromSmiles('OC(C1CCCCN1)c2cc(nc3c2cccc3C(F)(F)F)C(F)(F)F')]:
    if is_first_target:
        soft_reset = False
        is_first_target = False
    else:
        soft_reset = True
# for mol in [Chem.MolFromSmiles('O=C1CN=C(c2ccccn2)c2cc(Br)ccc2N1')]:
    try:
        smiles = Chem.MolToSmiles(mol)   
        print('expanding target {}'.format(smiles)) 
        #require starting materials small and cheap
        # status, paths = treeBuilder.get_buyable_paths(smiles, max_depth=10, template_prioritization=gc.relevance,
        #                                     precursor_prioritization=gc.scscore, nproc=1, expansion_time=300, max_trees=1000, 
        #                                     max_branching=25, apply_fast_filter=True, filter_threshold=0.9,
        #                                     max_ppg=1000,
        #                                     max_natom_dict={'C':10,'N':3,'O':3,'logic': 'and'},
        #                                     min_chemical_history_dict={'as_reactant':1, 'as_product':1, 'logic':'and'})
        status, paths = Tree.get_buyable_paths(smiles,
                                            nproc=NCPUS,
                                            max_depth=10,
                                            expansion_time=10,
                                            max_trees=1000, 
                                            max_cum_template_prob=0.995,
                                            max_branching=25, apply_fast_filter=True, filter_threshold=0.75,
                                            template_count=1000,
                                            max_natom_dict={'C':10,'N':3,'O':5,'logic': 'only'},
                                            # min_chemical_history_dict={'as_reactant':1, 'as_product':1,'logic':'and'},
                                            soft_reset=soft_reset,
                                            soft_stop=True)
        print('done for target {}'.format(smiles))

        # trees = [{'is_chemical': True, 'smiles': 'CN1C2CCC1CC(C2)OC(=O)C(CO)c3ccccc3', 'ppg': 0.0, 'id': 1, 'children': [{'info': '', 'smiles': 'CN1C2CCC1CC(O)C2.O=C(O)C(CO)c1ccccc1>>CN1C2CCC1CC(C2)OC(=O)C(CO)c3ccccc3', 'is_reaction': True, 'num_examples': 19578, 'template_score': 0.017628178000450134, 'children': [
        #     {'is_chemical': True, 'smiles': 'CN1C2CCC1CC(O)C2', 'ppg': 1.0, 'id': 3, 'children': []}, {'is_chemical': True, 'smiles': 'O=C(O)C(CO)c1ccccc1', 'ppg': 1.0, 'id': 4, 'children': []}], 'id': 2, 'necessary_reagent': u''}]}]
        feasible_trees = []
        for tree in paths:
            res = treeEvaluator.evaluate_tree(tree, gc.neural_network, gc.probability,
                               gc.templatefree, gc.forwardonly, is_target=True, reset=False, nproc=2, n=5)
            feasible_trees.append(res)
        routes_for_one_target = [{'chemicals':get_chemicals_from_tree(tree['tree'],path = []),'score':tree['score']} for tree in feasible_trees if tree['tree']['children']]
        context_for_one_target = [get_context_from_tree(tree['tree'],path = []) for tree in feasible_trees if tree['tree']['children']]
        # print(context_for_one_target)
        #index the route
        # index_list_one_target = range(len(paths))
        # print feasible_trees
        all_routes.extend(routes_for_one_target)
        all_routes_contexts.extend(context_for_one_target)
        all_trees.extend([tree for tree in feasible_trees if tree['tree']['children']])
        status_file.write('{}\t{}\t{}\t\n'.format(smiles,len(feasible_trees),''))
    except Exception as e:
        status_file.write('{}\t{}\t{}\t\n'.format(smiles,0,e))
        pass    # index_list_all_routes.extend(index_list_one_target)
    print(len(all_routes))


expanding target O=c1nc[nH]c2c1ncn2C1CCC(CO)O1
INFO@mcts_tree_builder   : [413.907s]	Doing a hard worker reset
INFO@mcts_tree_builder   : [414.257s]	Starting search for O=c1nc[nH]c2c1ncn2C1CCC(CO)O1
INFO@mcts_tree_builder   : [414.259s]	Preparing workers...
INFO@mcts_tree_builder   : [414.259s]	Tree builder spinning off 4 child processes
Model size: 50256K
INFO@mcts_tree_builder   : [415.301s]	Calculating initial probs for target
INFO@mcts_tree_builder   : [415.307s]	Set initial leaves for active pathways
INFO@mcts_tree_builder   : [415.308s]	Waiting for workers to initialize...
Model size: 50256K
Model size: 50256K
Model size: 50256K
INFO@mcts_tree_builder   : [417.310s]	Waiting for workers to initialize...
Loaded tf model from numpy arrays
Loaded tf model from numpy arrays
INFO@mcts_tree_builder   : [419.314s]	Waiting for workers to initialize...
Loaded tf model from numpy arrays
Loaded tf model from numpy arrays
INFO@mcts_tree_builder   : [421.321s]	Waiting for workers to initialize

/home/hanyug/Make-It/makeit/synthetic/evaluation/template_based_aux.py:88: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="tanh", kernel_regularizer=<keras.reg..., name="embed H_lost 1")`
  h_lost_h1    = Dense(N_h1, activation = inner_act, W_regularizer = l2(l2v), name = "embed H_lost 1")(h_lost_r)
/home/hanyug/Make-It/makeit/synthetic/evaluation/template_based_aux.py:89: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="tanh", kernel_regularizer=<keras.reg..., name="embed H_gain 1")`
  h_gain_h1    = Dense(N_h1, activation = inner_act, W_regularizer = l2(l2v), name = "embed H_gain 1")(h_gain_r)
/home/hanyug/Make-It/makeit/synthetic/evaluation/template_based_aux.py:90: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="tanh", kernel_regularizer=<keras.reg..., name="embed bond_lost 1")`
  bond_lost_h1 = Dense(N_h1, activation = inner_act, W_regularizer = l2(l2v), name = "embed bond_lost 1")

INFO@template_based      : [468.292s]	Scorer has been loaded.
INFO:tensorflow:Restoring parameters from /home/hanyug/Make-It/makeit/synthetic/evaluation/rexgen_direct/core_wln_global/model-300-3-direct/model.ckpt-140000
INFO:tensorflow:Restoring parameters from /home/hanyug/Make-It/makeit/synthetic/evaluation/rexgen_direct/rank_diff_wln/model-core16-500-3-max150-direct-useScores/model.ckpt-2400000
INFO@tree_evaluator      : [473.727s]	Evaluated reaction: O=C1c2cccc(O)c2C(=O)c2c(O)cccc21>>O=C1c2c(O)cccc2Cc2cccc(O)c21 - ranked 1 with a 48.7977431589% probability.
ERROR@evaluator          : [485.068s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [485.069s]	The expected outcome of O=C1OC(=O)c2c(O)cccc21.Oc1ccccc1 was O=C(c1ccc(O)cc1)c1cccc(O)c1C(=O)O (0.443321761117)
ERROR@evaluator          : [485.069s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [485.069s]	The expected outcome of O=C1OC(=O)c2c(O)cccc21.Oc1ccccc1 was O=C

INFO@tree_evaluator      : [556.754s]	Evaluated reaction: COc1cccc2c1Cc1c(cccc1OC)C2>>COc1cccc2c1C(=O)c1c(cccc1OC)C2 - ranked 2 with a 27.5435823534% probability.
ERROR@evaluator          : [560.668s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [560.669s]	The expected outcome of COc1cccc2c1C(=O)c1c(OC)cccc1C2=O was COc1cccc2c1C(=O)c1c(O)cccc1C2=O (0.570174353869)
ERROR@evaluator          : [560.670s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [560.670s]	The expected outcome of COc1cccc2c1C(=O)c1c(OC)cccc1C2=O was COc1cccc2c1C(=O)c1c(O)cccc1C2=O (0.936361703574)
ERROR@evaluator          : [560.670s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [560.670s]	The expected outcome of COc1cccc2c1C(=O)c1c(OC)cccc1C2=O was COc1cccc2c1C(=O)c1c(O)cccc1C2=O (0.936361703574)
ERROR@evaluator          : [560.671s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [560.6

ERROR@evaluator          : [580.827s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [580.827s]	The expected outcome of COc1cccc2cc3cccc(OC)c3cc12 was COc1cccc2c(O)c3cccc(OC)c3cc12 (0.970812885941)
ERROR@evaluator          : [580.828s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [580.828s]	The expected outcome of COc1cccc2cc3cccc(OC)c3cc12 was COc1cccc2c(O)c3cccc(OC)c3cc12 (0.991593365043)
ERROR@evaluator          : [580.828s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [580.829s]	The expected outcome of COc1cccc2cc3cccc(OC)c3cc12 was COc1cccc2c(O)c3cccc(OC)c3cc12 (0.970812885941)
ERROR@evaluator          : [580.829s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [580.829s]	The expected outcome of COc1cccc2cc3cccc(OC)c3cc12 was COc1cccc2c(O)c3cccc(OC)c3cc12 (0.926887487663)
ERROR@evaluator          : [580.829s]	Target not found in outcomes! Returning 0 

...

##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)Nc1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)Nc1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)Nc1ccc(cc1Br)S(=O)(=O)(=O)c1ccc(C)cc1
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N(S=O)c1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N(S=O)c1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)Nc1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)Nc1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N(S=O)c1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N(S=O)c1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N(c1ccc(cc1Br)S(=O)(=O)(=O)Cl)S(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N(c1ccc(cc1Br)S(=O)(=O)(=O)Cl)S(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)Nc1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)Nc1ccc(cc1Br)S(=O)(=O)(=O)Cl
##### Unparseable

##### Unparseable mol: Cc1onc2NS(=O)(=O)c3ccc(cc3)[N+](=O)([O-])C(O)c21
##### Unparseable mol: Cc1onc2NS(=O)(=O)c3ccc(cc3)[N+](=O)([O-])Cc21
##### Unparseable mol: CC(=O)OCC[N+]1(C(=O)[N+](=O)([O-])c2ccc(cc2)S1(=O)=O)c1cc(C)on1
##### Unparseable mol: OC[N+](=O)([O-])c1ccc(cc1)S(=O)(=O)Nc1cc(C=O)on1
##### Unparseable mol: CC(=O)OCC[N+]1(C[N+](=O)([O-])c2ccc(cc2)S1(=O)=O)c1cc(C)on1
##### Unparseable mol: CC[N+]1(C(=O)[N+](=O)([O-])c2ccc(cc2)S1(=O)=O)c1cc(C)on1
##### Unparseable mol: CC(OC(C)=O)[N+]1(C(=O)[N+](=O)([O-])c2ccc(cc2)S1(=O)=O)c1cc(C)on1
##### Unparseable mol: CC(=O)OC[N+]1(C(=O)[N+](=O)([O-])c2ccc(cc2)S1(=O)=O)c1cc(C)on1
##### Unparseable mol: Cc1cc(NS(=O)(=O)c2ccc(cc2)[N+](=O)([O-])C=O)no1
##### Unparseable mol: Cc1cc(NS(=O)(=O)c2ccc(cc2)[N+](=O)([O-])C=O)no1
##### Unparseable mol: CC(=O)[N+](=O)([O-])c1ccc(cc1)S(=O)(=O)Nc1cc(C)on1
##### Unparseable mol: CC(=O)[N+](=O)([O-])c1ccc(cc1)S(=O)(=O)Nc1cc(C)on1
##### Unparseable mol: CC(=O)OC(=O)[N+](=O)([O-])c1ccc(cc1)S(=O)(=O)Nc1c

INFO@tree_evaluator      : [1160.284s]	Evaluated reaction: Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)Nc3noc(C)c3Br)cc2)cc1>>Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)Nc3cc(C)on3)cc2)cc1 - ranked 2 with a 3.11557116667% probability.
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
##### Unparseable mol: Cc1onc(NS(=O)(=O)c2ccc([Na+])cc2)c1Br
INFO@tree_evaluator      : [1170.726s]	Evaluated reaction: Cc1ccc(S(=O)(=O)Cl)cc1.Cc1onc(NS(=O)(=O)c2ccc(N)cc2)c1Br>>Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)Nc3noc(C)c3Br)cc2)cc1 - ranked 1 with a 99.7425940151% probability.
INFO@tree_evaluator   

##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)NS(=O)(=O)(=O)c1ccc(I)cc1
##### Unparseable mol: O=S1(=O)(=O)Nc2ccc1cc2
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N(O)S(=O)(=O)(=O)c1ccc(I)cc1
##### Unparseable mol: NS(=O)(=O)(=O)c1ccc(I)cc1
##### Unparseable mol: ONS(=O)(=O)(=O)c1ccc(I)cc1
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N1Oc2ccc(cc2)S1(=O)(=O)=O
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)N1c2ccc(cc2)S1(=O)(=O)=O
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)c1cc(I)ccc1S(N)(=O)(=O)=O
##### Unparseable mol: Cc1ccc(cc1)S(=O)(=O)c1cc(I)ccc1S(=O)(=O)(=O)NO
ERROR@evaluator          : [1200.962s]	Target not found in outcomes! Returning 0 as score.
ERROR@evaluator          : [1200.962s]	The expected outcome of Cc1ccc(S(N)(=O)=O)cc1.O=S(=O)([O-])c1ccc(I)cc1 was  (0)
INFO@tree_evaluator      : [1200.963s]	Evaluated reaction: Cc1ccc(S(N)(=O)=O)cc1.O=S(=O)([O-])c1ccc(I)cc1>>Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)[O-])cc2)cc1 - ranked inf with a 0.0% probability.
INFO@tree_evaluator      : [1200.963s]

INFO@tree_evaluator      : [1260.070s]	Evaluated reaction: COC(=O)c1c(NS(=O)(=O)c2ccc(NS(=O)(=O)c3ccc(C)cc3)cc2)noc1C>>Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)Nc3noc(C)c3C(=O)O)cc2)cc1 - ranked 1 with a 99.8642508713% probability.
INFO@tree_evaluator      : [1276.360s]	Evaluated reaction: COC(=O)c1c(N)noc1C.Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)Cl)cc2)cc1>>COC(=O)c1c(NS(=O)(=O)c2ccc(NS(=O)(=O)c3ccc(C)cc3)cc2)noc1C - ranked 1 with a 99.6576017252% probability.
INFO@tree_evaluator      : [1276.362s]	Evaluated reaction: Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)[O-])cc2)cc1>>Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)Cl)cc2)cc1 - ranked 1 with a 99.7239376535% probability.
INFO@tree_evaluator      : [1276.362s]	Evaluated reaction: Cc1ccc(S(=O)(=O)Cl)cc1.Nc1ccc(S(=O)(=O)[O-])cc1>>Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)[O-])cc2)cc1 - ranked 1 with a 99.9717076128% probability.
INFO@tree_evaluator      : [1276.362s]	Found a fully plausible tree!
INFO@tree_evaluator      : [1276.363s]	Evaluated reaction: Cc1ccc(S(=O)(=O)Nc2ccc(S(=O)(

In [5]:
Tree.Chemicals

{'CC(=O)Cl': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Cl'),
 'CC(=O)Nc1ccc(S(=O)(=O)Cl)c(Br)c1': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Nc1ccc(S(=O)(=O)Cl)c(Br)c1'),
 'CC(=O)Nc1ccc(S(=O)(=O)Cl)c(Cl)c1': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Nc1ccc(S(=O)(=O)Cl)c(Cl)c1'),
 'CC(=O)Nc1ccc(S(=O)(=O)Cl)cc1': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Nc1ccc(S(=O)(=O)Cl)cc1'),
 'CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1'),
 'CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Cl)c1': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Cl)c1'),
 'CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1'),
 'CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1Br': <class 'makeit.retrosynthetic.mcts.nodes.Chemical'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)o

In [21]:
Tree.status

{('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 23): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 27): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 44): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 128): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 136): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 156): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 262): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 297): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 448): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 487): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 512): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 833): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 910): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 1570): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 2009): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 2821): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 2899): 1,
 ('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)c(Br)c1', 

In [18]:
chemical1 = Tree.Chemicals['CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1Br']
CTA = chemical1.template_idx_results[512]
print CTA

In [20]:
for template_idx in chemical1.template_idx_results:
    CTA = chemical1.template_idx_results[template_idx]
    if CTA.waiting or not CTA.valid:
          continue
    print(CTA.reactions)
    

{'CC(=O)OC(C)=O.Cc1cc(NS(=O)(=O)c2ccc(N)c(Br)c2)no1': <class 'makeit.retrosynthetic.mcts.nodes.Reaction'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1Br')}
{'CC(=O)Cl.Cc1cc(NS(=O)(=O)c2ccc(N)c(Br)c2)no1': <class 'makeit.retrosynthetic.mcts.nodes.Reaction'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1Br')}
{'Cc1cc(NS(=O)(=O)c2ccc(N)c(Br)c2)no1': <class 'makeit.retrosynthetic.mcts.nodes.Reaction'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1Br')}
{'CC(=O)OC(C)=O.Cc1cc(NS(=O)(=O)c2ccc([N+](=O)[O-])c(Br)c2)no1': <class 'makeit.retrosynthetic.mcts.nodes.Reaction'>('CC(=O)Nc1ccc(S(=O)(=O)Nc2cc(C)on2)cc1Br')}
